# Transfering SOL with `solana.py`

This notebook uses [Michael Huang's `solana.py`](https://github.com/michaelhly/solana-py) to,

1. Create a `Keypair`
2. Request an airdrop
3. Transfering some of it to a different `PublicKey`

That's not terribly interesting. But, it does show off the almost unreasonable effectiveness of effectiveness of executing transactions using blockchains.

## Setup

I'm assuming you know how to set up a python environment. You'll need to do a,

```bash
pip install solana
```

for the rest of this.

In [10]:
import asyncio

from solana.keypair import Keypair
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient
from solana.system_program import TransferParams, transfer
from solana.transaction import Transaction

## Step-by-Step

### Generate a Keypair

Crypto means cryptography! On the blockchain, too. First you'll need to generate a keypair. Basically, this means you're computer will need to generate some special random numbers. (Or, you can create one from a 32-byte seed.) Internally, `solana.py` uses [`NaCl`](https://nacl.cr.yp.to/) because "Don't Roll Your Own Crypto" is actually a much stronger best-practice constraint than "Don't Repeat Yourself". 

In [8]:
keypair = Keypair()

Rather than looking at the raw bytes, people share encoded representations that are easy to copy. [Base58](https://tools.ietf.org/id/draft-msporny-base58-01.html) is a common one. It doesn't have zero, capital-O, 1, or lowercase-L in it's alphabet. If you've ever tried to use these characters in a password, you'll understand why this is Good.

In [9]:
pk = keypair.public_key
print(f"Your BASE58-encoded public key: {pk}")

Your BASE58-encoded public key: D5UmnAErpKU8pFkxbJkeLwqE5pNavbdLgRMMwhQqafxd


### Create a client and connect to devnet

To mess around, we'll connect to [`devnet`](https://docs.solana.com/clusters#devnet). Nothing that happens here is real, and their is a faucet to give you free tokens (that are also not real in that they have no economic value).

In [11]:
client = AsyncClient("https://api.devnet.solana.com")
assert await client.is_connected()

### Check your Balance

In the first step you created a keypair. You can check the balance (0) of the account addressed by that public key.

In [4]:
await client.get_balance(pk)

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 95582119}, 'value': 0},
 'id': 1}

This is one of those (sorta) non-skeuomorphic things that confuses people a bit. 

If you have some SOL (or BTC, or ETH, or whatever) in your wallet, you don't "have" it in these sense you have a dollar in your wallet. Instead -- and the whole point -- is that you have a cryptographic object that can control value associated with its public key. But the data itself (**ignoring the private key**) resides entirely within the blockchain. There is no registration step or anything. So even though this script didn't send the public key to the cluster, you can query the associated balance.

### Request An Airdrop

Airdrops are a really *really* interesting part of how cryptocurrencies work. In this context though (`devnet`) it just gives you something that has no value for free so that you can fuck around. One `SOL` is 1,000,000,000 lamports. Crypo loves to make you squint to count the zeros.

In [14]:
ONE_SOL = 1_000_000_000

await client.request_airdrop(pk, ONE_SOL, commitment='finalized')

{'jsonrpc': '2.0',
 'result': '5zxvfWDhh2g3bZToodPo4arW2AKNDv2wqgu6kk6WAXSdnN4zPUVvUnUaWstCjRDMCabhuYELRvTBaYegepfPbu5W',
 'id': 2}

The requested [`commitment` parameter](https://docs.solana.com/implemented-proposals/commitment) (here as a keyword for legibility, which isn't required) is set to `finalized`. Remember, solana is decentralized (ish). You interacted with one node of the cluster. But that isn't the only one required for a transaction to be considered confirmed. 

### Check your balance, again

Look at that!

In [15]:
await client.get_balance(pk)

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 95590964}, 'value': 2000000000},
 'id': 3}

You've got SOL! Or, more concretely, the ledger says the account associated with your public key and controllable by the paired private one has a balance of 1 SOL or 1,000,000,000 lamports.

### Get account info

So what exactly is the account though? 

Well, the one that gave you is the [`System Program`](https://docs.solana.com/developing/runtime-facilities/programs#system-program) (`11111111111111111111111111111111`). If you've done programming on other block chains, consider this to be a contract. The account is not executable because you didn't use SOL to create a program.

In [16]:
await client.get_account_info(pk)

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 95591225},
  'value': {'data': ['', 'base64'],
   'executable': False,
   'lamports': 2000000000,
   'owner': '11111111111111111111111111111111',
   'rentEpoch': 221}},
 'id': 4}

### Now Create a Different account

This notebook just has you send some airdropped SOL to another account. So let's create one, bound to the variable `bob` for convieinece).

In [19]:
bob = Keypair.generate()
print(f"Bob's public key: {bob.public_key}")

Bob's public key: HbRNRW55h8Th6rG8RwKMr6DCgeHAaetWm2Rk7rMDK39P


### Transfer Some SOL

We want to share some of our massive valueless `devnet` token bounty with `bob`. To do so, we create a transaction that contains a transfer instruction, sign it, then send it to the cluster.

In [20]:
txn = Transaction()
instruction = transfer(TransferParams(from_pubkey=pk, to_pubkey=bob.public_key, lamports=500_000_000))
txn.add(instruction)
res = await client.send_transaction(txn, keypair)
txn_hash = res['result']
res

{'jsonrpc': '2.0',
 'result': '4QTEz6HqZg7mhjeGdoo5LhUqtCjD2Rsrf1zEtQSQ69j2uTSP7G1oyWKjjwquGus4HNNP1JQXCCGzxjYmKQ1xzy1H',
 'id': 6}

Same caveat as before applies. We didn't "register" Bob's address because that's not how any of this works.

### Check the Balance

Now let's make sure `Bob` got it.

In [22]:
await client.get_balance(bob.public_key)

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 95592289}, 'value': 954},
 'id': 8}

But wait! That's not the amount we wanted to send! Some of it got lost. The other part about crypto is you have to pay for transactions. The difference between what we sent and what was received is the gas. 

### View the Transaction

You don't __need__ to use the API to inspect this transaction, even though it's more fun. There are public explorers for that,

In [24]:
print(f"https://explorer.solana.com/tx/{txn_hash}?cluster=devnet")

https://explorer.solana.com/tx/4QTEz6HqZg7mhjeGdoo5LhUqtCjD2Rsrf1zEtQSQ69j2uTSP7G1oyWKjjwquGus4HNNP1JQXCCGzxjYmKQ1xzy1H?cluster=devnet


If you visit that link, you'll see,

```
Program 11111111111111111111111111111111 invoke [1]
Program 11111111111111111111111111111111 success
```

Which means that the transaction called the `System Program` successfully. The `System Program` does transfers.

### Cleanup

In [ ]:
await client.close()